## Получение картиночных эмбеддингов

В силу большого размера датасета обучение CV моделей непосредственно под задачу занимает много времени (в частности самая маленьккая версия EfficientNetV2 при размере батча 32 учится на Kaggle одну эпоху примерно в течение 1-2 часов). За неименееv более мощных ресурсов при таком времени обучения нет возможности тестировать гипотезы. В силу этого было решено использовать готовые модели для получения эмбеддингов. Была выбрана модель [SigLip2](https://huggingface.co/docs/transformers/model_doc/siglip2) - достаточно новая активно используемая модель, для получения эмбеддингов картинок. 

In [ ]:
import pandas as pd
from torchvision.io import read_image
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoProcessor

In [ ]:
DEVICE = "cuda"
train = pd.read_parquet(
    "/kaggle/input/characteristics-regression/train.parquet"
)
test = pd.read_parquet("/kaggle/input/characteristics-regression/test.parquet")

In [4]:
test_images = "/kaggle/input/characteristics-regression/test_images/test"
train_images = "/kaggle/input/characteristics-regression/train_images/train"

In [ ]:
# ckpt = "google/siglip2-base-patch16-224"
ckpt = "google/siglip2-large-patch16-256"  # Использование более большой модели дает несколько более информативные эмбеддинги
model = AutoModel.from_pretrained(ckpt).to(DEVICE).eval()
processor = AutoProcessor.from_pretrained(ckpt, use_fast=True)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        return read_image(self.image_paths[idx])

In [ ]:
batch_size = 32
train_dataset = ImageDataset(
    [f"{train_images}/{img}" for img in train["image_name"]]
)
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x
)

test_dataset = ImageDataset(
    [f"{test_images}/{img}" for img in test["image_name"]]
)
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x
)

In [ ]:
embeddings = []
for batch_images in tqdm(train_dataloader):
    inputs = processor(images=batch_images, return_tensors="pt").to(
        model.device
    )
    with torch.no_grad():
        batch_embeds = model.get_image_features(**inputs)

    embeddings.append(batch_embeds)

embeddings = torch.cat(embeddings, dim=0)

100%|██████████| 9779/9779 [2:12:19<00:00,  1.23it/s]  


In [ ]:
torch.save(embeddings, "embeddings/train_embeddings_large.pt")

In [ ]:
embeddings = []
for batch_images in tqdm(test_dataloader):
    inputs = processor(images=batch_images, return_tensors="pt").to(
        model.device
    )
    with torch.no_grad():
        batch_embeds = model.get_image_features(**inputs)

    embeddings.append(batch_embeds)

embeddings = torch.cat(embeddings, dim=0)

100%|██████████| 2197/2197 [29:45<00:00,  1.23it/s]


In [ ]:
torch.save(embeddings, "embeddings/test_embeddings_large.pt")